In [1]:
import pandas as pd
import nltk
import string

In [2]:
fragebogen = pd.read_csv('./fragebogen.csv', names=["id",
                                                    "nummer", 
                                                    "titel", 
                                                    "schlagwoerter", 
                                                    "erscheinungsjahr",
                                                    "autoren", 
                                                    "originaldaten",
                                                    "anmerkung",
                                                    "freigabe",
                                                    "checked",
                                                    "wordleiste",
                                                    "druck",
                                                    "online",
                                                    "publiziert",
                                                    "fragebogen_typ_id",])

fragebogen.drop(["id","schlagwoerter","erscheinungsjahr","autoren","originaldaten","anmerkung","freigabe",
                 "checked","wordleiste","druck","online","publiziert","fragebogen_typ_id",], inplace=True, axis=1)

fragebogen.set_index("nummer", drop=True, inplace=True)

In [3]:
fragebogen.info()

<class 'pandas.core.frame.DataFrame'>
Index: 762 entries, 1 to ETir.28
Data columns (total 1 columns):
titel    762 non-null object
dtypes: object(1)
memory usage: 11.9+ KB


In [7]:
fragebogen.head()

,titel
nummer,
1,Fragebogen 1: Kopf (1)
2,Fragebogen 2: Die Osterwoche (1)
3,Fragebogen 3: Die Osterwoche (2)
4,Fragebogen 4: Kopf (2)
5,Fragebogen 5: Zeit zwischen Ostern und Fronlei...


In [12]:
fragebogen.titel.str.extract(r'([Fragebon]+)\s{1}([0-9]+)[:]{1}([A-ZÄÓÜa-zäöü0-9\s]+)[,\s]*([\(\)A-ZÄÓÜa-zäöü0-9]*)').head()

,0,1,2,3
nummer,,,,
1,Fragebogen,1,Kopf,(1)
2,Fragebogen,2,Die Osterwoche,(1)
3,Fragebogen,3,Die Osterwoche,(2)
4,Fragebogen,4,Kopf,(2)
5,Fragebogen,5,Zeit zwischen Ostern und Fronleichnam,


In [58]:
fragebogen['number'] = \
fragebogen.titel.str.extract(r'([Fragebon]+)\s{1}([0-9]+)[:]{1}([,A-ZÄÖÜa-zäöüß0-9.\s]+)[,\s]*([=\-\(\)\sA-ZÄÖÜa-zäöüß0-9]*)')[1]

fragebogen['headwords'] = \
fragebogen.titel.str.extract(r'([Fragebon]+)\s{1}([0-9]+)[:]{1}([,A-ZÄÖÜa-zäöüß0-9.\s]+)[,\s]*([=\-\(\)\sA-ZÄÖÜa-zäöüß0-9]*)')[2]

fragebogen['series'] = \
fragebogen.titel.str.extract(r'([Fragebon]+)\s{1}([0-9]+)[:]{1}([,A-ZÄÖÜa-zäöüß0-9.\s]+)[,\s]*([=\-\(\)\sA-ZÄÖÜa-zäöüß0-9]*)')[3]

In [59]:
fragebogen.head()

,titel,number,headwords,series
nummer,,,,
1,Fragebogen 1: Kopf (1),1,Kopf,(1)
2,Fragebogen 2: Die Osterwoche (1),2,Die Osterwoche,(1)
3,Fragebogen 3: Die Osterwoche (2),3,Die Osterwoche,(2)
4,Fragebogen 4: Kopf (2),4,Kopf,(2)
5,Fragebogen 5: Zeit zwischen Ostern und Fronlei...,5,Zeit zwischen Ostern und Fronleichnam,


In [64]:
print(fragebogen.number.unique())
print()
print(fragebogen.headwords.unique())
print()
print(fragebogen.series.unique())

['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29' '30'
 '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43' '44'
 '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56' '57' '58'
 '59' '60' '61' '62' '63' '64' '65' '66' '67' '68' '69' '70' '71' '72'
 '73' '74' '75' '76' '77' '78' '79' '80' '81' '82' '83' '84' '85' '86'
 '87' '88' '89' '90' '91' '92' '93' '94' '95' '96' '97' '98' '99' '100'
 '101' '102' '103' '104' '105' '106' '107' '108' '109' nan]

[' Kopf ' ' Die Osterwoche ' ' Zeit zwischen Ostern und Fronleichnam'
 ' Menschl. Haar und Bart ' ' Hochzeit ' ' Nase, Ohr ' ' Auge '
 ' Besiedelung, Flur ' ' Feld, Feldbestellung ' ' Körper ' ' Körperteile '
 ' Hautkrankheiten ' ' Brotbacken ' ' Weißgebäck' ' Gliedmaßen, Arm'
 ' Arm ' ' Hand ' ' Finger ' ' Schneiderei ' ' Kleidung ' ' Bewegung '
 ' Farbe' ' Zeit zwischen Mittsommer und Neujahr' ' Tod'
 ' Gruß, Wunsch, Bitte und Dank' '

In [62]:
#print(fragebogen[fragebogen.number == '104']['titel'])
print(fragebogen[fragebogen.series == '(2) (Allg']['titel'])

nummer
17    Fragebogen 17: Feld, Feldbestellung (2) (Allg.)
Name: titel, dtype: object


In [66]:
fragebogen.head()

,titel,number,headwords,series
nummer,,,,
1,Fragebogen 1: Kopf (1),1,Kopf,(1)
2,Fragebogen 2: Die Osterwoche (1),2,Die Osterwoche,(1)
3,Fragebogen 3: Die Osterwoche (2),3,Die Osterwoche,(2)
4,Fragebogen 4: Kopf (2),4,Kopf,(2)
5,Fragebogen 5: Zeit zwischen Ostern und Fronlei...,5,Zeit zwischen Ostern und Fronleichnam,
